# Learn-to-Rank Keras Example

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_ranking as tfr
import glob

In [2]:
tf.__version__

'2.5.0'

In [3]:
tfr.__version__

'0.4.0.dev'

### Keras example

https://github.com/tensorflow/ranking/blob/master/tensorflow_ranking/examples/keras/keras_dnn_tfrecord.py

In [4]:
from typing import Dict, Tuple
from absl import app
from absl import flags
import tensorflow as tf
import tensorflow_ranking as tfr

In [5]:
# Arguments
num_features = 136

# The document relevance label.
_LABEL_FEATURE = "utility"

# Padding labels are set negative so that the corresponding examples can be
# ignored in loss and metrics.
_PADDING_LABEL = -1
_MASK = "example_list_mask"

def _create_feature_spec() -> Tuple[Dict[str, tf.io.FixedLenFeature], Dict[
    str, tf.io.FixedLenFeature], Tuple[str, tf.io.FixedLenFeature]]:
    """Create context and example feature spec for data parsing.
    Returns:
    (context feature specs, example feature specs, label spec).
    """
    context_feature_spec = {}
    example_feature_spec = {
      "custom_features_{}".format(i + 1):
      tf.io.FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=0.0)
      for i in range(0, num_features)
    }
    label_spec = (_LABEL_FEATURE,
                tf.io.FixedLenFeature(
                    shape=(1,), dtype=tf.int64, default_value=_PADDING_LABEL))
    return context_feature_spec, example_feature_spec, label_spec

In [6]:
context_feature_spec, example_feature_spec, label_spec = _create_feature_spec()

In [7]:
context_feature_spec

{}

In [8]:
# example_feature_spec

In [9]:
example_feature_spec['custom_features_1']

FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=0.0)

In [10]:
label_spec

('utility', FixedLenFeature(shape=(1,), dtype=tf.int64, default_value=-1))

In [11]:
from tensorflow_ranking.python.keras.pipeline import DatasetHparams

In [12]:
# Arguments
train_input_pattern = "/data/train_numerical_elwc.tfrecord"
valid_input_pattern = "/data/vali_numerical_elwc.tfrecord"
train_batch_size = 32
valid_batch_size = 32
list_size = None
convert_labels_to_binary = False

# Get dataset hyperparams
dataset_hparams = tfr.python.keras.pipeline.DatasetHparams(
    train_input_pattern=train_input_pattern,
    valid_input_pattern=valid_input_pattern,
    train_batch_size=train_batch_size,
    valid_batch_size=valid_batch_size,
    list_size=list_size,
    dataset_reader=tf.data.TFRecordDataset,
    convert_labels_to_binary=convert_labels_to_binary)

In [13]:
dataset_hparams

DatasetHparams(train_input_pattern='/data/train_numerical_elwc.tfrecord', valid_input_pattern='/data/vali_numerical_elwc.tfrecord', train_batch_size=32, valid_batch_size=32, list_size=None, valid_list_size=None, dataset_reader=<class 'tensorflow.python.data.ops.readers.TFRecordDatasetV2'>, convert_labels_to_binary=False)

In [14]:
# dir(dataset_hparams)

In [15]:
# Arguments
model_dir = "/tmp/output" # f"/outputs/tf-ranking-{datetime.now().strftime('%d-%m-%Y-%H-%M-%S')}"
num_epochs = 10
num_train_steps = 1000
num_valid_steps = 100
loss = "approx_ndcg_loss"
optimizer = "adagrad"
learning_rate = 0.005
steps_per_execution = 10
export_best_model = False
strategy = "MirroredStrategy"

pipeline_hparams = tfr.keras.pipeline.PipelineHparams(
      model_dir=model_dir,
      num_epochs=num_epochs,
      steps_per_epoch=(num_train_steps // num_epochs),
      validation_steps=num_valid_steps,
      loss=loss,
      loss_reduction=tf.losses.Reduction.AUTO,
      optimizer=optimizer,
      learning_rate=learning_rate,
      steps_per_execution=steps_per_execution,
      export_best_model=export_best_model,
      strategy=strategy)

In [16]:
pipeline_hparams

PipelineHparams(model_dir='/tmp/output', num_epochs=10, steps_per_epoch=100, validation_steps=100, learning_rate=0.005, loss='approx_ndcg_loss', loss_reduction='auto', optimizer='adagrad', loss_weights=None, steps_per_execution=10, automatic_reduce_lr=False, use_weighted_metrics=False, export_best_model=False, best_exporter_metric_higher_better=False, best_exporter_metric='loss', strategy='MirroredStrategy', tpu='')

In [17]:
# Arguments
use_log1p = False

preprocess_dict = {}
if use_log1p:
    preprocess_dict = {
        fname: lambda t: tf.math.log1p(t * tf.sign(t)) * tf.sign(t)
        for fname in example_feature_spec.keys()
    }

In [18]:
hidden_layer_dims = "64,32,16"
use_batch_norm = True
batch_norm_moment = 0.99
dropout = 0.4

dnn_scorer = tfr.keras.model.DNNScorer(
    hidden_layer_dims=map(int, hidden_layer_dims.split(",")),
    output_units=1,
    activation=tf.nn.relu,
    input_batch_norm=use_batch_norm,
    use_batch_norm=use_batch_norm,
    batch_norm_moment=batch_norm_moment,
    dropout=dropout)

In [19]:
dnn_scorer

In [20]:
model_builder = tfr.keras.model.ModelBuilder(
    input_creator=tfr.keras.model.FeatureSpecInputCreator(
      context_feature_spec, 
        example_feature_spec),
    preprocessor=tfr.keras.model.PreprocessorWithSpec(preprocess_dict),
    scorer=dnn_scorer,
    mask_feature_name=_MASK,
    name="keras_dnn_model")

In [21]:
model_builder

In [22]:
# dir(model_builder)

In [23]:
ranking_pipeline = tfr.keras.pipeline.SimplePipeline(
    model_builder=model_builder,
    dataset_builder=tfr.keras.pipeline.SimpleDatasetBuilder(
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        mask_feature_name=_MASK,
        label_spec=label_spec,
        hparams=dataset_hparams),
    hparams=pipeline_hparams)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [24]:
ranking_pipeline.train_and_validate(verbose=1)

Epoch 1/10
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


100/100 [==============================] - 29s 287ms/step - loss: -0.7261 - metric/ndcg_1: 0.5347 - metric/ndcg_5: 0.6705 - metric/ndcg_10: 0.7346 - metric/ndcg: 0.7346 - val_loss: -0.7841 - val_metric/ndcg_1: 0.7408 - val_metric/ndcg_5: 0.8744 - val_metric/ndcg_10: 0.8978 - val_metric/ndcg: 0.8978
Epoch 2/10
100/100 [==============================] - 13s 129ms/step - loss: -0.7436 - metric/ndcg_1: 0.5668 - metric/ndcg_5: 0.6923 - metric/ndcg_10: 0.7523 - metric/ndcg: 0.7523 - val_loss: -0.8016 - val_metric/ndcg_1: 0.6299 - val_metric/ndcg_5: 0.8309 - val_metric/ndcg_10: 0.8544 - val_metric/ndcg: 0.8544
Epoch 3/10
100/100 [==============================] - 9s 92ms/step - loss: -0.7469 - metric/ndcg_1: 0.5967 - metric/ndcg_5: 0.6981 - metric/ndcg_10: 0.7562 - metric/ndcg: 0.7562 - val_loss: -0.7935 - val_metric/ndcg_1: 0.5188 - val_metric/ndcg_5: 0.7914 - val_metric/ndcg_10: 0.8068 - val_metric/ndcg: 0.8068
Epoch 4/10
100/100 [==============================] - 10s 103ms/step - loss: -0.

INFO:tensorflow:Assets written to: /tmp/output/export/latest_model/assets
